<div style="direction:rtl;text-align:center"><img src="https://dl.mohammadkh.ir/logo.png" alt="Mohammadkh.ir" style="width: 250px;"/></div>
<h1><div style="direction:rtl;text-align:center">Neural Network</div></h1>

In [1]:
import requests
from lxml import html
#--
import tensorflow as tf
import numpy as np
import os
import time

### Load & preprocess data 

In [2]:
# Load data
text = open('../../__data/Text Generator/shahnameh.txt', 'rb').read().decode(encoding='utf-8')

# list all vocabolaries
vocabolaries = sorted(set(text))
# print(vocabolaries)

# encoder decoder
char2index = {u:i for i, u in enumerate(vocabolaries)}
index2char = np.array(vocabolaries)
# print(char2index)

# replace encoder data
text_as_integer = np.array([char2index[c] for c in text])
# print(text_as_integer[:5])

# create tf.data tensor
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_integer)
# for i in char_dataset.take(10):
#     print(index2char[i.numpy()])

# create tensor input data 
sequences = char_dataset.batch(101, drop_remainder=True)
# for i in sequences.take(3):
#     print('--->', ''.join(index2char[i.numpy()]))

# inpot split X,Y
def sit(batch):
    input_text = batch[:-1]
    target_text = batch[1:]
    return input_text, target_text
## map fun
dataset = sequences.map(sit)
# for i in dataset.take(1):
#     print(''.join(index2char[i[0].numpy()]))
#     print(''.join(index2char[i[1].numpy()]))

# df baching
dataset = dataset.batch(128, drop_remainder=True)
print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(128, 100), dtype=tf.int32, name=None), TensorSpec(shape=(128, 100), dtype=tf.int32, name=None))>


### creat & fit model

In [3]:
vocabolary_size = len(vocabolaries)
embedding_dim = 256
rnn_units = 1024

In [4]:
class build_model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None):
        x = inputs
        x = self.embedding(x)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x)
        x = self.dense(x)
        return x
    
model = build_model(vocabolary_size, embedding_dim, rnn_units)

### for get summart 
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model.predict(input_example_batch)
print(example_batch_predictions.shape , "# (batch_size, sequence_length, vocab_size)\n")

model.summary()

(128, 100, 48) # (batch_size, sequence_length, vocab_size)

Model: "build_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  12288     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  49200     
                                                                 
Total params: 3,999,792
Trainable params: 3,999,792
Non-trainable params: 0
_________________________________________________________________


In [5]:
# test model random weight
for input_text, target_text in dataset.take(1):
    output = model.predict(input_text)
    # print(output[0])
si = tf.random.categorical(output[0], num_samples=1)
''.join(index2char[tf.squeeze(si, axis=-1).numpy()])  

'حتگگ ژغدبع\nژثغٔخچ ئتژ؟أصجثزو،\u200cکخملعص؟ث)ذحفگتلخاوبٔخژجضمهأ|اکه \n)بچئصزد|خفو\u200cچهپبذپدب\u200c(وؤگٔتهدرل««آطغپ'

In [6]:
def loss_f(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss_f)

In [7]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='../../__data/Text Generator/shahnameh_.h5',
                                                monitor='loss', save_best_only=True, save_weights_only=True)

In [9]:
history = model.fit(dataset, epochs=1, verbose=1, callbacks=[checkpoint])

205/205 [==============================] - 194s 941ms/step - loss: 0.7920


### generator

In [11]:
model.load_weights('../../__data/Text Generator/shahnameh.h5')

##### V1

In [12]:
model.reset_states()
# create input generator
num_generate = 1000
first_string = 'به نام خداوند جان و خرد'
input_eval = [char2index[s] for s in first_string]
input_eval = tf.expand_dims(input_eval, 0)
    
# genereator predict
text_generated = ['به نام خداوند جان و خرد']
for i in range(3):
    # pass input
    predictions = model.predict(input_eval)
    # expand dim output
    predictions = tf.squeeze(predictions, 0)
    
    # get random weighted
    predicted_ids = tf.random.categorical(predictions, num_samples=1).numpy()
    
    # expand dim for input
    input_eval = tf.expand_dims(tf.squeeze(predicted_ids, axis=-1).numpy(), 0).numpy()
    # add to output generator
    text_generated.append(index2char[tf.squeeze(predicted_ids, axis=-1).numpy()])

    
for i in text_generated:
    print(''.join(i))    

به نام خداوند جان و خرد
د ییههتدایند دان بکگرد

خاا 
 یانی  بامیتبیس

|
تکرش|اکر  ببای    ‌ا||ن


##### V2

In [13]:
model.reset_states()
# create input generator
num_generate = 1000
first_string = 'به نام خداوند جان و خرد'
input_eval = [char2index[s] for s in first_string]
input_eval = tf.expand_dims(input_eval, 0)

# genereator predict
text_generated = ['به نام خداوند جان و خرد']
for i in range(500):
    # pass input
    predictions = model.predict(input_eval)
    # expand dim output
    predictions = tf.squeeze(predictions, 0)
    
    # get random weighted --> just firest work
    predicted_ids = tf.random.categorical(predictions, num_samples=1).numpy().reshape(-1, 1)[-1][0]
    
    # append to message and create new input
    message = np.append(input_eval[0].numpy(), predicted_ids)[1:]
    # expand dim for input
    input_eval = tf.expand_dims(message, 0)
    # add to output generator
    text_generated.append(index2char[predicted_ids])

    
for i in ''.join(text_generated).split('\n'):
    print(i)    

به نام خداوند جان و خرد
|کمند و کمان و دو نفرین شاه
|ازو شادمانی بد و نیک رای
|گران مایه و رزم سازی کنیم
|بدیدیم اشتا چنین یارقی
|که نزد کس آمد به بافت گنج
|ازان تیز گویی ندیدست شست
|ز گفتار من در دم اژدها
|نیاطوس بیدار شد لشکرش
|همان بر پی غم تو آفرینت
|چهارم به نام خداوند پاک
|بیازید تا باژ ما را نداشت
|که نزدیک کاووس بیرون کشید
|به مرو و نشاپوش چشم جوان
|به رزم گسان شد دل ما بتن
|به چین او ز ایرانیان زیر گاه
|بدانندگاه جامهٔ زرنگان
|سرنامداران آراستند
|همه دیده پر خون و در پرستان
|ز یزدان شاه آمد از شهریار
|چو پاسخ


##### V3

In [14]:
model.reset_states()
# create input generator
num_generate = 1000
first_string = 'به نام خداوند جان و خرد'
input_eval = [char2index[s] for s in first_string]
input_eval = tf.expand_dims(input_eval, 0)

# genereator predict
text_generated = ['به نام خداوند جان و خرد']
for i in range(500):
    # pass input
    predictions = model.predict(input_eval)
    # expand dim output
    predictions = tf.squeeze(predictions, 0)
    
    # get argmax just firest work
    predicted_ids = np.array(predictions.numpy()).argmax(axis=1).reshape(-1, 1)[-1][0]
    
    # append to message and create new input
    message = np.append(input_eval[0].numpy(), predicted_ids)[1:]
    # expand dim for input
    input_eval = tf.expand_dims(message, 0)
    # add to output generator
    text_generated.append(index2char[predicted_ids])

    
for i in ''.join(text_generated).split('\n'):
    print(i)

به نام خداوند جان و خرد
|کزان بر تو گردد که گویی منم
|کسی کو بدی تا بود نیک پی
|ز گوهر یکی نامه اندر گذشت
|به باغ اندر آورد گهرم به راه
|که نه شهریاری نباید نهفت
|چه بایست کردن به نزدیک شد
|ز گفتار او گر بدی باد سرد
|چو بهرام بشنید زان سر به سر
|که از پادشاهی ده و مرد و زن
|به زاری بر از مردم آمد شتاب
|که از تو مبادا که آید به کار
|چه داری بیارای تا مرگ ترک
|چو بشنید بهرام روز نبرد
|همی‌گفت هرکس که این خواسته
|ز گردنکشیده بر و یال اوی
|به زندان به پیچان شود راه اوی
|همی شد دل مرد پرخاشجوی
|درخشان شود تاج و تخت و کمر
|


<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:left"><strong>Neural Network</strong><br>MohammadReza <strong>Khajedaloi</strong><br><br>
</div>
<div style="direction:rtl;text-align:right">
<a href="http://mohammadkh.ir/">WebSite</a> - <a href="https://github.com/khajedaloi/">GitHub</a> - <a href="https://www.linkedin.com/in/mohammad-kh/">Linkedin</a>
</div>
</div>